In [1]:

from tarski.theories import Theory
from tarski.io.fstrips import ParsingError, FstripsReader
from tarski.syntax import *

# Parsing and Processing PDDL

In [2]:
reader = FstripsReader(raise_on_error=True)

In [3]:
reader.parse_domain('probBLOCKS-domain.pddl')
reader.parse_instance('probBLOCKS-4-2.pddl')

In [4]:
L = reader.problem.language

We can retrieve any sort (or _type_ in PDDL-parlance), constant, function or predicate 
via the `get` method of the class `Language`. Since we are dealing
with the classic `Blocks World` benchmark we know there must be predicates `on(x,y)` and `clear(x)` 

In [5]:
on = L.get('on')
clear = L.get('clear')

`Language` instances allow us to iterate over the list of declared predicates etc. via the
properties `predicates`, `functions`

In [6]:
for pred in L.predicates:
    print(str(pred))

=/2
!=/2
on/2
ontable/1
clear/1
handempty/0
holding/1


As seen above `Tarski` parser defines the equality predicate and its negation, as  well as the
usual `on` etc. The number after the name of the predicate indicates its _arity_ (i.e. number of arguments).

Getting our hands on the objects defined in the instance file `probBLOCKS-4-2.pddl` requires to 
navigate the type hierarchy. First, we can take a look at what are the sorts defined similarly as
we do for predicates

In [7]:
for S in L.sorts:
    print(str(S))

Sort(object)


There is only one sort defined, `object`. We can check out its elements 

In [8]:
objects = L.get('object')
for obj in objects.domain():
    print(str(obj), type(obj))

d <class 'tarski.syntax.terms.Constant'>
a <class 'tarski.syntax.terms.Constant'>
b <class 'tarski.syntax.terms.Constant'>
c <class 'tarski.syntax.terms.Constant'>


### Excursion: Indexing Logical Elements

It is handy to have the objects indexed in a table for easy retrieval. We can have such a table
by name

In [9]:
obj_table1 = { str(obj): obj for obj in objects.domain()}
obj_table1

{'a': a (object), 'b': b (object), 'c': c (object), 'd': d (object)}

Alternatively, we can use the constants themselves as keys in a dictionary, by wrapping them with
an adapter in this way

In [10]:
obj_table2 = { symref(obj): [] for obj in objects.domain()}
obj_table2

{symref[a]: [], symref[b]: [], symref[c]: [], symref[d]: []}

Wrapping constant, terms and atoms is necessary as `Tarski` overloads most of the standard Python 
operators to provide compact syntax for formulas and expressions. For instance, if we wanted
to compare whether two constants are the same _value_ one could try the following

In [11]:
x = L.get('a')
y = L.get('b')
x == y

=(a,b)

Yet using the comparison operator `==` results in the _atom_ `=(a,b)`, as `a` and `b` stand for
two elements of a first-order language. If we use the wrappers

In [12]:
x = L.get('a')
y = L.get('b')
symref(x) == symref(y)

False

Then we get to compare the actual _physical_ objects than the _logical elements_ (e.g.
 constants, arithmetic expressions, Boolean formulas) they represent.

## Grounding    

`Tarski` supports two different strategies for grounding lifted representations of STRIPS and ADL. One is based around
the idea by Malte Helmert of constructing a logic program and computing its models so as to determine the reachability
of a ground atom _online_. This is strategy constrasts with the classical approach, which we refer to as `naive`, where 
all potential groundings are first enumerated and then tested by reachability. While Helmert's approach is slower
for smaller problems, it does pay off greatly for instances with thousands of ground actions, or more, which can only
rarely be processed even by optimized implementations of the `naive` approach.

To implement Helmert's algorithm, or `lp` from now on, we rely on the excellent tools developed by for Answer Set
Programming, and inparticular, `gringo`, the suite of grounding tools for
disjunctivel logic programs. Stable versions of tools are neatly packaged in Ubuntu 18.04 so you can install them
like this

```
$ sudo apt install gringo
```

We can get hold of the classes encapsulating the grounding process from the `tarski.grounding` module

In [13]:
from tarski.grounding import LPGroundingStrategy
from tarski.grounding.errors import ReachabilityLPUnsolvable

and ground the instance of Blocks World with a one-liner

In [14]:
try:
    grounding = LPGroundingStrategy(reader.problem)
except ReachabilityLPUnsolvable:
    print("Problem was determined to be unsolvable during grounding")

Note that the `lp` grounder can determine if an instance is unsolvable, so we need to handle that possible exception.

In [15]:
actions = grounding.ground_actions()

for name, ops in actions.items():
    print('Action schema', name, 'got', len(ops), 'ground actions')

Action schema pick-up got 4 ground actions
Action schema put-down got 4 ground actions
Action schema stack got 16 ground actions
Action schema unstack got 16 ground actions


In [16]:
lpvariables = grounding.ground_state_variables()
for atom_index, atoms in lpvariables.enumerate():
    print('p_{}:'.format(atom_index), atoms)

p_0: on(a,a)
p_1: on(c,c)
p_2: on(c,b)
p_3: on(c,a)
p_4: on(a,c)
p_5: on(d,a)
p_6: on(d,c)
p_7: on(d,b)
p_8: on(a,b)
p_9: on(b,b)
p_10: on(d,d)
p_11: on(c,d)
p_12: on(a,d)
p_13: on(b,c)
p_14: on(b,a)
p_15: on(b,d)
p_16: ontable(d)
p_17: ontable(c)
p_18: ontable(a)
p_19: ontable(b)
p_20: handempty()
p_21: clear(d)
p_22: clear(a)
p_23: clear(c)
p_24: clear(b)
p_25: holding(d)
p_26: holding(a)
p_27: holding(c)
p_28: holding(b)
